# automate threshold finding

In [18]:
import numpy as np
import scipy.optimize
import scipy.interpolate
from py_vlasov.util import kzkp
from KSpaceSearch import KThetaScan

import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import pandas as pd

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.minor.size'] = 3.5
mpl.rcParams['ytick.minor.size'] = 3.5
mpl.rcParams['xtick.major.width'] = 1.1
mpl.rcParams['ytick.major.width'] = 1.1
mpl.rcParams['xtick.minor.width'] = .8
mpl.rcParams['ytick.minor.width'] = .8
mpl.rcParams['legend.fontsize'] = 14
#xtick.major.size     : 3.5      # major tick size in points
#xtick.minor.size     : 2      # minor tick size in points
#xtick.major.width    : 0.8    # major tick width in points
#xtick.minor.width    : 0.6    # minor tick width in points
%matplotlib inline

In [29]:
# aol = np.sqrt(2.65e-8)
aol = np.sqrt(1e-8)
print('aol = ', aol)
tc_tp = 1.5
th_tc = 4
nc_np = 0.95
nh_np = 1 - nc_np
# vc_va = 2.42
vc_va = 5
vh_va = -vc_va * nc_np / nh_np
print('vh_va = {0:.3g}'.format(vh_va))

# betap = 0.25
betap = .5
theta = 80


t_list=[1., tc_tp, th_tc * tc_tp]
a_list=[1., 1., 1.]
n_list=[1., nc_np, nh_np] 
q_list=[1.,-1., -1.]
m_list=[1., 1./1836., 1./1836]
v_list=[0., vc_va, vh_va]
n = 6
method = 'pade'

k = 0.2

target_k = 2

kz, kp = kzkp(k, theta)

param = [kz, kp, betap, t_list, a_list, n_list, q_list,
         m_list, v_list, n, method, aol]

aol =  0.0001
vh_va = -95


In [30]:
old_result_db_file = 'data/alfven_instability_thtc4.csv'
column_list = ['betap', 'vcva', 'tctp', 'thtc', 'vac', 
               'max_growth_rate', 'kz_peak', 'kp_peak', 'theta_peak']
if os.path.exists(old_result_db_file):
    old_df = pd.read_csv(old_result_db_file)
else:
    old_df = pd.DataFrame(columns= column_list)

In [31]:
result_db_file = 'data/alfven_instability_tctp15_thtc4.csv'
column_list = ['betap', 'vcva', 'tctp', 'thtc', 'vac', 
               'max_growth_rate', 'kz_peak', 'kp_peak', 'theta_peak']
if os.path.exists(result_db_file):
    df = pd.read_csv(result_db_file)
else:
    df = pd.DataFrame(columns= column_list)

In [32]:
def genParam(betap, vcva, param):
    newParam = param.copy()
    newParam[2] = betap
    n_list = param[5]
    vhva = - vcva * n_list[1] / n_list[2]
    newParam[8] = [0, vcva, vhva]
    return newParam

def autoSearch(betap, vcva, param, df, result_db_file='', refThtc=4, lookupTable=old_df):
    column_list = ['betap', 'vcva', 'tctp', 'thtc', 'vac', 
               'max_growth_rate', 'kz_peak', 'kp_peak', 'theta_peak']
    if os.path.exists(result_db_file):
        df = pd.read_csv(result_db_file)
    else:
        df = pd.DataFrame(columns= column_list)
    if len(df[(df.betap==betap)&(df.vcva==vcva)]) > 0:
        return df
    print('working on betap = {}, vcva = {}'.format(betap, vcva))
    new_param = genParam(betap, vcva, param)
    cond_1 = lookupTable.betap == betap
    cond_2 = lookupTable.vcva == vcva
    cond_3 = lookupTable.thtc == refThtc
    row = lookupTable[cond_1 & cond_2 & cond_3]
    theta_peak = row.theta_peak.values[0]
    kz_peak = row.kz_peak.values[0]
    kp_peak = row.kp_peak.values[0]
    k_peak = np.sqrt(kz_peak**2 + kp_peak**2)
    
    isValid = False
    while not isValid:
        theta_min = theta_peak - 5
        theta_max = theta_peak + 5
        n_theta = int(np.ceil(theta_max - theta_min)+1) 
        if n_theta < 12:
            n_theta *= 2
        theta_arr = np.linspace(theta_min, theta_max, n_theta)

        if k_peak < 0.5:
            k_min = k_peak / 2
            lin_incrmt = 0.015
        elif k_peak < 1.2:
            k_min = 0.3
            lin_incrmt = 0.02
        else:
            lin_incrmt = 0.04
            k_min = 0.4

        k_max = k_peak * 1.3

        alfven_stop_condition = lambda guess, wrel: (wrel.real > 1) or (wrel.real <= 0) or (wrel.imag < -.5) or (wrel.real < 0.3 * guess)
        ktscan = KThetaScan(new_param, 
                            lin_incrmt=lin_incrmt, 
                            incrmt_method='linear', 
                            gridNum = 500,
                            stop_condition=alfven_stop_condition, 
                            diagnostic_dir = 'alfven_instability_diagnostics/', 
                            savefig = False)

        kz_2d, kp_2d, k_2d, wrel_2d = ktscan.fullScan(theta_arr, k_min, k_max)

        ktscan.makeDiagnostics1D(theta_arr, k_2d, wrel_2d, makeplot=False)

        ktscan.makeDiagnostics2D(theta_arr, kz_2d, kp_2d, wrel_2d, makeplot=False)
        theta_peak = ktscan.maxGrowthTheta
        k_peak = np.sqrt(ktscan.maxGrowthK[0]**2 + ktscan.maxGrowthK[0]**2)
        if np.abs(theta_peak - theta_min) < 1 or \
            np.abs(theta_peak - theta_max) < 1:
            print('solution not in range. Adjust searching range.')
            continue
        isValid = True
        
        

    data_dict = {'betap': betap, 'vcva': vcva, 'tctp': tc_tp, 'thtc': th_tc, 'vac': aol, 
                 'max_growth_rate': ktscan.maxGrowthRate, 
                 'kz_peak': ktscan.maxGrowthK[0], 
                 'kp_peak': ktscan.maxGrowthK[1], 
                 'theta_peak': ktscan.maxGrowthTheta}
    new_row = pd.DataFrame(data=data_dict, columns=column_list, index=[0])
    df = df.append(new_row)

    print("max growth rate = {0:.2g}".format(ktscan.maxGrowthRate))

    df = df.sort_values(['betap', 'vcva'], ascending=[True, True])

    df.to_csv(result_db_file, float_format='%.3g', index=False)
    return df


In [41]:
for betap in old_df.betap.unique():
    vcva_arr = old_df[old_df.betap == betap].vcva.unique()
    for vcva in vcva_arr:
        new_df = autoSearch(betap, vcva, param, df, 
                            result_db_file=result_db_file, 
                            refThtc=4, lookupTable=old_df)

working on betap = 0.1, vcva = 1.0
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 1.4e-05
working on betap = 0.1, vcva = 1.2
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.00047
working on betap = 0.1, vcva = 1.3
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.00088
working on betap = 0.1, vcva = 1.4
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0014
working on betap = 0.1, vcva = 1.8
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.005
working on betap = 0.1, vcva = 2.0
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0075
working on betap = 0.1, vcva = 2.2
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.01
working on betap = 0.1, vcva = 2.4
No user provided wave mode guess function. Use default Alfven 

/Users/Tong/Envs/science/lib/python3.6/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


max growth rate = 0.014
working on betap = 0.3, vcva = 2.2
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0016
working on betap = 0.3, vcva = 2.3
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0024
working on betap = 0.3, vcva = 2.4
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0033
working on betap = 0.3, vcva = 2.5
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0041
working on betap = 0.3, vcva = 2.6
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0051
working on betap = 0.3, vcva = 3.0
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.0092
working on betap = 0.3, vcva = 3.4
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.014
working on betap = 0.3, vcva = 3.6
No user provided wave mode guess functi

max growth rate = 0.026
working on betap = 1.0, vcva = 6.6
No user provided wave mode guess function. Use default Alfven guess.
max growth rate = 0.028


In [39]:
old_df[old_df.betap == 0.05]

,betap,vcva,tctp,thtc,vac,max_growth_rate,kz_peak,kp_peak,theta_peak
0,0.05,1.0,2,4,0.0001,0.000226,0.142,0.269,62.2
1,0.05,1.2,2,4,0.0001,0.001180,0.137,0.469,73.7
2,0.05,1.4,2,4,0.0001,0.003000,0.132,0.620,78.0
3,0.05,1.8,2,4,0.0001,0.008660,0.124,0.828,81.5
4,0.05,2.0,2,4,0.0001,0.012200,0.119,0.928,82.7


In [36]:
new_df

,betap,vcva,tctp,thtc,vac,max_growth_rate,kz_peak,kp_peak,theta_peak
0,0.05,1.0,1.5,4,0.0001,0.000250,0.141000,0.319000,66.100000
1,0.05,1.2,1.5,4,0.0001,0.001370,0.137000,0.542000,75.800000
2,0.05,1.4,1.5,4,0.0001,0.003440,0.131000,0.725000,79.700000
3,0.05,1.8,1.5,4,0.0001,0.009580,0.123000,0.973000,82.800000
0,0.05,2.0,1.5,4,0.0001,0.013271,0.119733,1.038808,83.425098


In [38]:
result_db_file = 'data/alfven_instability_tctp15_thtc4.csv'
column_list = ['betap', 'vcva', 'tctp', 'thtc', 'vac', 
               'max_growth_rate', 'kz_peak', 'kp_peak', 'theta_peak']
if os.path.exists(result_db_file):
    df = pd.read_csv(result_db_file)
else:
    df = pd.DataFrame(columns= column_list)
df

,betap,vcva,tctp,thtc,vac,max_growth_rate,kz_peak,kp_peak,theta_peak
0,0.05,1.0,1.5,4,0.0001,0.00025,0.141,0.319,66.1
1,0.05,1.2,1.5,4,0.0001,0.00137,0.137,0.542,75.8
2,0.05,1.4,1.5,4,0.0001,0.00344,0.131,0.725,79.7
3,0.05,1.8,1.5,4,0.0001,0.00958,0.123,0.973,82.8
4,0.05,2.0,1.5,4,0.0001,0.01330,0.120,1.040,83.4
